<a href="https://colab.research.google.com/github/dmswl0707/RecomendationSystem_Python/blob/main/01_content_based_filtering_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [45]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/RCS_data/01data.csv')
data.head(2)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,budget,genres,homepage,id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,keywords
0,9,58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,en,GoldenEye,James Bond must unmask the mysterious head of ...,14.686036,"[{'name': 'United Artists', 'id': 60}, {'name'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,6.6,1194.0,"[{'id': 701, 'name': 'cuba'}, {'id': 769, 'nam..."
1,24,3600000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.mgm.com/title_title.do?title_star=L...,451,en,Leaving Las Vegas,"Ben Sanderson, an alcoholic Hollywood screenwr...",10.332025,"[{'name': 'United Artists', 'id': 60}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-27,49800000.0,112.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,I Love You... The Way You Are.,Leaving Las Vegas,7.1,365.0,"[{'id': 30, 'name': 'individual'}, {'id': 549,..."


In [46]:
del data['Unnamed: 0']

In [47]:
df = data
df.head(2)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4402 entries, 0 to 4401
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4402 non-null   int64  
 1   genres                4402 non-null   object 
 2   homepage              4402 non-null   object 
 3   id                    4402 non-null   int64  
 4   original_language     4402 non-null   object 
 5   original_title        4402 non-null   object 
 6   overview              4402 non-null   object 
 7   popularity            4402 non-null   float64
 8   production_companies  4402 non-null   object 
 9   production_countries  4402 non-null   object 
 10  release_date          4402 non-null   object 
 11  revenue               4402 non-null   float64
 12  runtime               4402 non-null   float64
 13  spoken_languages      4402 non-null   object 
 14  status                4402 non-null   object 
 15  tagline              

In [48]:
###최종으로 사용할 데이터 (이전 메타데이터에서 실습데이터로 전처리)
df = df[['id','genres', 'vote_average', 'vote_count','popularity','title',  'keywords', 'overview']]
df.copy()

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
0,710,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",6.6,1194.0,14.686036,GoldenEye,"[{'id': 701, 'name': 'cuba'}, {'id': 769, 'nam...",James Bond must unmask the mysterious head of ...
1,451,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",7.1,365.0,10.332025,Leaving Las Vegas,"[{'id': 30, 'name': 'individual'}, {'id': 549,...","Ben Sanderson, an alcoholic Hollywood screenwr..."
2,807,"[{'id': 80, 'name': 'Crime'}, {'id': 9648, 'na...",8.1,5915.0,18.457430,Se7en,"[{'id': 476, 'name': 'self-fulfilling prophecy...",Two homicide detectives are on a desperate hun...
3,629,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",8.1,3334.0,16.302466,The Usual Suspects,"[{'id': 3703, 'name': 'law'}, {'id': 5493, 'na...","Held in an L.A. interrogation room, Verbal Kin..."
4,10634,"[{'id': 35, 'name': 'Comedy'}]",7.0,513.0,14.569650,Friday,"[{'id': 897, 'name': 'rap music'}, {'id': 970,...",Craig and Smokey are two guys in Los Angeles h...
...,...,...,...,...,...,...,...,...
4397,60153,"[{'id': 35, 'name': 'Comedy'}, {'id': 99, 'nam...",7.5,2.0,0.084820,Hi Way,"[{'id': 1462, 'name': 'samurai'}, {'id': 1859,...","Two young aspiring filmmakers, director Jaco a..."
4398,383618,"[{'id': 12, 'name': 'Adventure'}, {'id': 10751...",0.0,0.0,2.660047,Siv Sleeps Astray,"[{'id': 1370, 'name': 'shapeshifting'}, {'id':...",Seven-year-old Siv’s first night sleeping over...
4399,103462,[],0.0,0.0,0.017109,Grandma Lo-Fi,[],"At the tender age of 70 years, Sigrídur Níelsd..."
4400,432789,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",6.2,37.0,5.667067,The Incredible Jessica James,"[{'id': 1419, 'name': 'gun'}, {'id': 1938, 'na...","Burned by a bad breakup, a struggling New York..."


# 평가 기준 조정 
**영화 평점 vote_average에 대한 이슈**
투표 결과가 많을 수록 평점이 낮아지는 이슈에 대한 해결 

**( v / (v+m) * R ) + (m / (m + v) * C)** 을 이용한다.

- r : 개별 영화 평점
- v : 개별 영화에 평점을 투표한 횟수
- m : 500위 안에 들어야 하는 최소 투표
- c : 전체 영화에 대한 평균 평점


**영화 후보군을 정하기 위한 상위선 기준**
- 탑 500위 영화들은 상위 몇 퍼센트 영화인가? 답은 89%
(상위 88%는 529회이기 때문에 89퍼센트으로 가정한다)

#데이터 집계하기(C = column)
- df.describe() = 컬럼 value에 대한 info
- df['C'].quantile(퍼센트) = 백분율 값 리턴
- df.grouby(['C']).count() = 건수
- df.grouby(['C']).mean() = 평균
- df.grouby(['C']).sum() = 합계
- df.grouby(['C']).std() = 표준편차


---
###더 알아보기
df.groupby(['C']) : 중복된 row 값으로 그룹별 value 리턴
df.groupby(['지역'])['판매'].transform('count/mean/sum/std') = 지역별 판매 (집계 종류)



In [49]:
# 백분율 88% 득표 값
tmp_m = df['vote_count'].quantile(0.88)
tmp_m 

1410.88

In [68]:
#df에서 투표값이 1410.8보다 같거나 크다면 tmp_data로 정의한다
tmp_data = df.copy().loc[df['vote_count'] >= tmp_m]
print(tmp_data)
tmp_data.shape #4402 중에서 529개가 있다

          id  ...                                           overview
2        807  ...  Two homicide detectives are on a desperate hun...
3        629  ...  Held in an L.A. interrogation room, Verbal Kin...
5        755  ...  Seth Gecko and his younger brother Richard are...
13        11  ...  Princess Leia is captured and held hostage by ...
19      8587  ...  A young lion cub named Simba can't wait to be ...
...      ...  ...                                                ...
4252  337339  ...  When a mysterious woman seduces Dom into the w...
4312  324852  ...  Gru and his wife Lucy must stop former '80s ch...
4327  281338  ...  Caesar and his apes are forced into a deadly c...
4345  374720  ...  The miraculous evacuation of Allied soldiers f...
4362  335988  ...  Autobots and Decepticons are at war, with huma...

[529 rows x 8 columns]


(529, 8)

In [69]:
#상위 89퍼센트 총 485위 데이터
m = df['vote_count'].quantile(0.89)

#Fdata = Final Data
Fdata = df.loc[df['vote_count'] >= m]
Fdata.head()
Fdata.info()

C = Fdata['vote_average'].mean()
print(C)
print(m)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 485 entries, 2 to 4345
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            485 non-null    int64  
 1   genres        485 non-null    object 
 2   vote_average  485 non-null    float64
 3   vote_count    485 non-null    float64
 4   popularity    485 non-null    float64
 5   title         485 non-null    object 
 6   keywords      485 non-null    object 
 7   overview      485 non-null    object 
dtypes: float64(3), int64(1), object(4)
memory usage: 34.1+ KB
6.839999999999998
1547.6699999999996


In [70]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    
    return ( v / (v+m) * R ) + (m / (m + v) * C)

#새로운 집계 column 추가
df['C'] = df.apply(집계 종류, axis)

In [71]:
#Weight Score 완성
Fdata['score'] = Fdata.apply(weighted_rating, axis = 1)

Fdata.head(5)
#Fdata.info()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
2,807,"[{'id': 80, 'name': 'Crime'}, {'id': 9648, 'na...",8.1,5915.0,18.457430,Se7en,"[{'id': 476, 'name': 'self-fulfilling prophecy...",Two homicide detectives are on a desperate hun...,7.838691
3,629,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",8.1,3334.0,16.302466,The Usual Suspects,"[{'id': 3703, 'name': 'law'}, {'id': 5493, 'na...","Held in an L.A. interrogation room, Verbal Kin...",7.700533
5,755,"[{'id': 27, 'name': 'Horror'}, {'id': 28, 'nam...",6.9,1644.0,15.339153,From Dusk Till Dawn,"[{'id': 246, 'name': 'dancing'}, {'id': 380, '...",Seth Gecko and his younger brother Richard are...,6.870905
13,11,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",8.1,6778.0,42.149697,Star Wars,"[{'id': 803, 'name': 'android'}, {'id': 4270, ...",Princess Leia is captured and held hostage by ...,7.865777
19,8587,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",8.0,5520.0,21.605761,The Lion King,"[{'id': 2143, 'name': 'loss of parents'}, {'id...",A young lion cub named Simba can't wait to be ...,7.745985


In [72]:
Fdata.shape

(485, 9)

In [73]:
#딕셔너리 value 정리하기
Fdata[['genres', 'keywords']].head(2)

,genres,keywords
2,"[{'id': 80, 'name': 'Crime'}, {'id': 9648, 'na...","[{'id': 476, 'name': 'self-fulfilling prophecy..."
3,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...","[{'id': 3703, 'name': 'law'}, {'id': 5493, 'na..."


##from ast import literal_eval
- df.apply(liter_eval) : syntax를 구조화 해주는 함수

In [74]:
Fdata['genres'] = Fdata['genres'].apply(literal_eval)
Fdata['keywords'] = Fdata['keywords'].apply(literal_eval)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [75]:
Fdata[['genres', 'keywords']].head(2)

,genres,keywords
2,"[{'id': 80, 'name': 'Crime'}, {'id': 9648, 'na...","[{'id': 476, 'name': 'self-fulfilling prophecy..."
3,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...","[{'id': 3703, 'name': 'law'}, {'id': 5493, 'na..."


In [76]:
#name 컬럼만 추출
Fdata['genres'] = Fdata['genres'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))
Fdata['keywords'] = Fdata['keywords'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [78]:
Fdata.head(2)

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
2,807,Crime Mystery Thriller,8.1,5915.0,18.457430,Se7en,self-fulfilling prophecy detective s.w.a.t. dr...,Two homicide detectives are on a desperate hun...,7.838691
3,629,Drama Crime Thriller,8.1,3334.0,16.302466,The Usual Suspects,law relatives theft criminal criminal mastermi...,"Held in an L.A. interrogation room, Verbal Kin...",7.700533


In [80]:
Fdata.to_csv('/content/drive/My Drive/Colab Notebooks/data/RCS_data/01data_2.csv', index = False)
